In [94]:
# load core modules
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
# load agents and tools modules
import pandas as pd
from io import StringIO
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

# initialize pinecone client and connect to pinecone index
pinecone.init(      
	api_key='65cd1f0b-fa6c-42dc-902e-5be9acffa7c5',      
	environment='us-west1-gcp-free'      
)      
index = pinecone.Index('tk-policy')

# initialize embeddings object; for use with user query/input
embed = OpenAIEmbeddings(
                model = 'text-embedding-ada-002'
            )

# initialize langchain vectorstore(pinecone) object
text_field = 'text' # key of dict that stores the text metadata in the index
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

llm = ChatOpenAI(    
    model_name="gpt-3.5-turbo", 
    temperature=0.0
    )

# initialize vectorstore retriever object
timekeeping_policy = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

df = pd.read_csv("data/employee_data.csv") # load employee_data.csv as dataframe
python = PythonAstREPLTool(locals={"df": df}) # set access of python_repl tool to the dataframe

# create calculator tool
calculator = LLMMathChain.from_llm(llm=llm, verbose=True)

# create variables for f strings embedded in the prompts
user = 'Alexander Verdad' # set user
df_columns = df.columns.to_list() # print column names of df

# prep the (tk policy) vectordb retriever, the python_repl(with df access) and langchain calculator as tools for the agent
tools = [
    Tool(
        name = "Timekeeping Policies",
        func=timekeeping_policy.run,
        description="""
        Useful for when you need to answer questions about employee timekeeping policies.

        <user>: What is the policy on unused vacation leave?
        <assistant>: I need to check the timekeeping policies to answer this question.
        <assistant>: Action: Timekeeping Policies
        <assistant>: Action Input: Vacation Leave Policy - Unused Leave
        ...
        """
    ),
    Tool(
        name = "Employee Data",
        func=python.run,
        description = f"""
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: {df_columns}
        
        <user>: How many Sick Leave do I have left?
        <assistant>: df[df['name'] == '{user}']['sick_leave']
        <assistant>: You have n sick leaves left.              
        """
    ),
    Tool(
        name = "Calculator",
        func=calculator.run,
        description = f"""
        Useful when you need to do math operations or arithmetic.
        <user>: How much will I be paid if I encash my unused VLs?
        <assistant>: df[df['name'] == '{user}'][['basic_pay_in_php', 'vacation_leave']]
        <assistant>: You will be paid Php n if you encash your unused VLs.'
        """
    )
]

# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {'prefix': f'You are friendly HR assistant. You are tasked to assist the current user: {user} on questions related to HR. You have access to the following tools:'}


# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         agent_kwargs=agent_kwargs
                         )
# define q and a function for frontend
def get_response(user_input):
    response = agent.run(user_input)
    return response

In [89]:
print(agent.agent.llm_chain.prompt.template)

You are friendly HR assistant. You are tasked to assist the current user: Alexander Verdad on questions related to HR. You have access to the following tools:

Timekeeping Policies: 
        Useful for when you need to answer questions about employee timekeeping policies.

        <user>: What is the policy on unused vacation leave?
        <assistant>: I need to check the timekeeping policies to answer this question.
        <assistant>: Action: Timekeeping Policies
        <assistant>: Action Input: Vacation Leave Policy - Unused Leave
        ...
        
Employee Data: 
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: ['employee_id', 'name', 'position', 'organizational_unit', 'rank', 'hire_date', 'regularization_date', 'vacation_leave', 'sick_leave', 'basic_pay_in_php', 'employment_status', 'supervisor']
   

In [90]:
get_response("How many Sick Leave do I have left?")



> Entering new  chain...
I need to check the employee data to find the sick leave balance for Alexander Verdad.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad']['sick_leave']
Observation: 2    8
Name: sick_leave, dtype: int64
Thought:The sick leave balance for Alexander Verdad is 8.
Final Answer: You have 8 sick leaves left.

> Finished chain.


'You have 8 sick leaves left.'

In [91]:
get_response("What position is me?")



> Entering new  chain...
I need to check the employee data to find the position of Alexander Verdad.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad']['position']
Observation: 2    Account Recievable Assistant
Name: position, dtype: object
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Sun, 09 Jul 2023 00:39:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e3c7b2aff96bea7-CGK', 'alt-svc': 'h3=":443"; ma=86400'}.


The position of Alexander Verdad is Account Receivable Assistant.
Final Answer: Account Receivable Assistant

> Finished chain.


'Account Receivable Assistant'

In [95]:
get_response("How much will I be paid if I encash my unused VLs?")



> Entering new  chain...
I need to check the employee data to find the basic pay and the number of unused vacation leaves.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad'][['basic_pay_in_php', 'vacation_leave']]
Observation:    basic_pay_in_php  vacation_leave
2             15000              45
Thought:I have found that Alexander Verdad has a basic pay of Php 15,000 and 45 unused vacation leaves.
Thought: I can now calculate the amount to be paid for encashing the unused VLs.
Action: Calculator
Action Input: 15000 * 45

> Entering new  chain...
15000 * 45```text
15000 * 45
```
...numexpr.evaluate("15000 * 45")...

Answer: 675000
> Finished chain.

Observation: Answer: 675000
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TnwHydAcTkdNqsyKHl7UVDMq on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TnwHydAcTkdNqsyKHl7UVDMq on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

The amount to be paid for encashing the unused VLs is Php 675,000.
Final Answer: Php 675,000

> Finished chain.


'Php 675,000'

In [96]:
get_response('Are employees on probation allowed to have vacation leaves?')



> Entering new  chain...
I need to check the timekeeping policies to answer this question.
Action: Timekeeping Policies
Action Input: Vacation Leave Policy - Probationary Employees
Observation: During probation, employees are not eligible for Vacation Leave.
Thought:I now know the final answer
Final Answer: During probation, employees are not eligible for Vacation Leave.

> Finished chain.


'During probation, employees are not eligible for Vacation Leave.'

In [97]:
df = pd.read_csv("data/employee_data.csv")
df.head()

,employee_id,name,position,organizational_unit,rank,hire_date,regularization_date,vacation_leave,sick_leave,basic_pay_in_php,employment_status,supervisor
0,1005674,Joseph Peña,Accounts Recievable Supervisor,Finance Department,Supervisor,2/16/2013,7/16/2013,30,15,20000,Permanent,Richard Santos
1,1005675,Mark Delos Santos,HR Assistant,HR Department,Rank and File,1/1/2023,NaN,20,2,15000,Probation,Jinky Francisco
2,1005676,Alexander Verdad,Account Recievable Assistant,Finance Department,Rank and File,11/1/2017,5/1/2018,45,8,15000,Permanent,Joseph Santos
3,1005677,Jinky Francisco,Recruitment Supervisor,HR Department,Supervisor,10/15/2021,4/15/2022,16,4,15000,Permanent,NaN
4,1005678,Richard Santos,Accounts Recievable Head,Finance Department,Manager,3/16/2016,9/16/2016,50,11,30000,Permanent,NaN
